In [19]:
import os
import torch
import random
import pandas as pd

from training import train_model
from testing import test_model
from model import CharLSTM
from utils import set_seed
from dataset import SmilesDataset

In [14]:
set_seed(0)

data_path = '/media/Molecules/molecules.csv'

train_size = 1_000_000
val_size = 100_000

path_to_save = "./saved_weights/attempt_3"

n_epochs = 30
batch_size = 64
lr = 1e-3

cuda_device = 2
n_jobs = 3

In [15]:
data = pd.read_csv(data_path, usecols=['SMILES'], nrows=(train_size + val_size), squeeze=True)
data = data.tolist()

random.shuffle(data)
train_data = data[:train_size]
val_data = data[train_size: (train_size + val_size)]

train_dataset = SmilesDataset(train_data)
val_dataset = SmilesDataset(val_data)

In [17]:
train_model(train_dataset, val_dataset, path_to_save, n_epochs, batch_size, lr, n_jobs, cuda_device)

In [20]:
char_rnn = CharLSTM(hidden_size=32, num_layers=1, train_dataset=train_dataset, val_dataset=val_dataset)
char_rnn = char_rnn.to(torch.device('cuda:{device}'.format(device=cuda_device)))
char_rnn.load_state_dict(torch.load(os.path.join(path_to_save, "epoch_best.pt")))

test_model(char_rnn, val_dataset)

Your vocabulary:
['#', '(', ')', '+', '-', '1', '2', '3', '4', '5', '6', '7', '8', '<EOS>', '<SOS>', '=', 'B', 'C', 'F', 'H', 'I', 'N', 'O', 'P', 'S', '[', ']', 'c', 'l', 'n', 'o', 'p', 'r', 's']

###########################
Internal Diversity = 0.10180943047234481
External Diversity = 0.12431580941089086
Validity = 0.423
Uniqueness = 1.0
############################

